# PHY 3310 - Quantum Mechanics
# Stern-Gerlach Experiment using a Quantum Computer
-----------------------------------------------------------

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from qiskit import *
#%matplotlib inline

In the last activity, we introduced some basic aspects of how to use quantum computers to simulate a single spin-$\frac{1}{2}$ degree of freedom. Of particular importance were:

1. An arbitrary two-level state can be created from the default initialization $\left|+\right\rangle$ by application of $\hat{U}(\theta,\phi,\lambda)$ for suitable choices of $\theta$, $\phi$ with $\lambda = 0$.
2. The spin projection of a state can be measured in any direction $\hat{n}(\theta,\phi)$ by appropriately rotating the state before performing a standard measurement of $\hat{S}^{z}$ (i.e., the "computational basis").

In today's endeavor, we will build on these ideas to construct a simulation of the famous Stern-Gerlach experiment. The term "simulation" refers to the actual quantum processes taking place on transmon qubits rather than spins, to be distinguished from the classical "simulator" used by the [SPINS](http://sites.science.oregonstate.edu/~mcintyre/ph425/spins/) program to generate sample data.

To streamline the process somewhat, let us begin by defining several measurement functions. The goal is to be able to <i>use</i> a bunch of quantum circuits to get results. The manual construction of gates associated with spin component measurments becomes quite tedious. It will be assumed that a circuit has been defined of the form

```
qr = QuantumRegister(nq)
cr = ClassicalRegister(nc)
circuit = QuantumCircuit(qr,qc)
```

Given this (empty) circuit, the qubits will always be initialized to $\left|+\right\rangle$. Our task will be to perform successive measurements of various spin components. The following functions will create such measurements:

In [ ]:
def measure_sx(circuit,qreg,creg):
    circuit.ry(-np.pi/2,qreg)
    circuit.measure(qreg,creg)
    circuit.ry(np.pi/2,qreg)
    return circuit

def measure_sy(circuit,qreg,creg):
    circuit.rz(-np.pi/2,qreg)
    circuit.ry(-np.pi/2,qreg)
    circuit.measure(qreg,creg)
    circuit.ry(np.pi/2,qreg)
    circuit.rz(np.pi/2,qreg)
    return circuit

def measure_sz(circuit,qreg,creg):
    circuit.measure(qreg,creg)
    return circuit

<b>Important warning: </b> The above functions will rotate the system, perform the measurement, and then <i>un-rotate</i> the system. This is perhaps the easiest conceptual way to approach repeated measurements. However, it's far from the most efficient approach. One could design each compound circuit from scratch to reduce the number of gates. For our purposes (and certainly with simulator results), this shouldn't matter much. But if you want to do quantum computing for a living, you'll want to pay much more attention to doing things with as few gates as possible. 

With that disclaimer out of the way, let's get going and take a single spin represented by a single qubit. Implicitly, the initialization of the qubit in the state $\left|0\right\rangle$ represents a measurement of $\hat{S}^{z}$ on an unpolarized beam followed by discarding all the spins in state $\left|-\right\rangle$.

So here's a simple task: imagine we wish to perform the following sequence of measurements (screenshot from [SPINS Java app](http://sites.science.oregonstate.edu/~mcintyre/ph425/spins/)):

![axes](sg1a.png)

It's important to note that the IBM quantum hardware essentially takes care of the first measurement and only feeds us the $\left|+\right\rangle$ states which emerge from the first analyzer. Of course it doesn't actually generate the states this way, but for purposes of simulation we can pretend that's what's happening. 

Our job is just to complete the circuit, which just involves measuring $\hat{S}^{x}$:

In [ ]:
qr = QuantumRegister(1)
cr = ClassicalRegister(1)

circ = QuantumCircuit(qr,cr)
circ = measure_sx(circ,0,0)
circ.draw()

Recall that the measurement process takes the state of qubit and writes it to the classical register, which is just a blank set of slots (also initialized to zero). When we begin taking multiple measurements, we'll need to write to different slots on the classical register to record all the measurements. 

But for now, this should do it. Let's see what happens. For simplicity, we'll run the simulator and an actual device and show both sets of results on the same histogram. 

In [ ]:
IBMQ.load_account() 

In [ ]:
job = execute(circ,backend=BasicAer.get_backend('qasm_simulator'),shots=1024)
counts = job.result().get_counts()

qcomp = provider.get_backend('ibmq_bogota')
job = execute(circ,backend=qcomp)

from qiskit.tools.monitor import job_monitor
job_monitor(job)

counts_q = job.result().get_counts()

In [ ]:
from qiskit.tools.visualization import plot_histogram
plot_histogram([counts,counts_q],legend=['simulator','device'])

Recall that we need to interpet '0' or '1' as '+' or '-' <i>in this basis for which the measurment is performed</i>. We conclude that of all the spins initialized in $\left|+\right\rangle$, about half were measured to be $\left|+\right\rangle_{x}$ while the other half were measured to be in $\left|-\right\rangle$. So far nothing too magical.
    
Surely, measuring $\hat{S}^{z}$ one more time would return $+$ for all measurements since the state began in that eigenstate. We can explore this by adding another measurement. Since this will require a larger classical register to store two measurements, we'll just redo the circuit:

In [ ]:
qr = QuantumRegister(1)
cr = ClassicalRegister(2)

circ = QuantumCircuit(qr,cr)
circ = measure_sx(circ,0,1)
circ = measure_sz(circ,0,0)
circ.draw()

One potentially confusing item (at least for me): It appears the the classical register state appears written right-to-left. I tend to think left-to-right in most cases, so I have written the first measurement to ```cr[1]``` and the second to ```cr[0]```. It doesn't matter which measurement gets stored where, but (as of the time of writing this) this appears be necessary to get an output that can be read from left to right (but more on this below!). For completion, here is the Stern-Gerlach setup being used:

<img src="sg2.png" alt="sg2" width="600"/>

As before, the initialization on the backend takes care of the first $\hat{S}^{z}$ measurement. Note that the Stern-Gerlach experiment only feeds the $\left|+\right\rangle_{x}$ into the final analyzer. There is nothing in our quantum circuit that makes this restriction, but we will be able to account for this once we get the counts--the joys of post-processing!

So let's run this thing and see what happens:

In [ ]:
job = execute(circ,backend=BasicAer.get_backend('qasm_simulator'),shots=1024)
counts = job.result().get_counts()

qcomp = provider.get_backend('ibmq_bogota')
job = execute(circ,backend=qcomp)

from qiskit.tools.monitor import job_monitor
job_monitor(job)

counts_q = job.result().get_counts()

In [ ]:
plot_histogram([counts,counts_q],legend=['simulator','device'])

Crudely, it looks like about a quarter of the initial $\left|+\right\rangle$ spins go into each state. There's a bit to unpack here, so let's do it carefully.

The classical register stores the outcomes ('0' or '1' in qubit language corresponding to '+' or '-' in spin language) of a single measurement. Our "counts" tell us all of the classical register states and how many times each was observed. 

Let's start with '00'. In the spin language, this is '++'. But the register doesn't record the measurement direction, so we have to reintroduce that information when interpreting what it means. Recalling that we begin with a state $\left|+\right\rangle$, we measured $\hat{S}^{x}$ and then measured $\hat{S}^{z}$. Thus, the state labeled by '00' means the measurment of $\hat{S}^{x}$ returned +$\frac{\hbar}{2}$, projecting the system into the eigenstate $\left|+\right\rangle_{x}$, and the subsequent measurement of $\hat{S}^{z}$ resulted in $+\frac{\hbar}{2}$. Since the initialization <i>gave</i> us a state with $S^{z} = + \frac{\hbar}{2}$, this all seems quite reasonable.

But the <i>funny</i> business concerns the existence of states '01' and '11'. The second register element being '1' indicates that the $\hat{S}^{z}$ measurement returned $-\frac{\hbar}{2}$. This is bizarre because it's like observing that an apple is red, tasting it to find that it is sweet, and then looking to see that it is green. Well that's a sloppy analogy, but no analogy is perfect.


Let's dive in to examine this more deeply. In the following exercises, you're going to make some simple modifications to the circuits presented above.

<b>Exercise: </b> Perhaps a quantum measurement simply kicks the state randomly. Reconfigure the circuit to simply measure $\hat{S}^{z}$ twice. Interpret your result in words.

<b>Exercise: </b> Instead of $\hat{S}^{z}$, try measuring $\hat{S}^{x}$ twice. Are the results the same as the previous case? Why or why not?

<b>Exercise</b>: Open the SPINS program and run the following simulation:

<img src="spins1.png" alt="spins1" width="600"/>

Now construct a quantum circuit to simulate this case. Note that the Stern-Gerlach setup only <i>analyzes</i> the $\hat{S}^{x}$ states without recording a measurement. You can mimic this by modifying the function ```measure_sx()``` to ```analyze_sx()``` as follows:

```
def analyze_sx(circuit,qreg,creg):
    circuit.ry(-np.pi/2,qreg)
    #circuit.measure(qreg,creg)
    circuit.ry(np.pi/2,qreg)
    return circuit
```

This is sloppy programming, but it emphasizes that all you're changing is to delete the actual measurement. The rotation (and anti-rotation) still takes place and represents the "analyzing" done in the SPINS software. 

Interpret your results.